<img src="https://github.com/gantian127/nwis/blob/master/docs/source/_static/nwis_logo.png?raw=true" width='320' align='center'></a>

## Basic Info
**Please note that nwis package is deprecated.**

**Please check out [bmi_nwis](https://github.com/gantian127/bmi_nwis) package** for 
Basic Model Interface ([BMI](https://bmi-spec.readthedocs.io/en/latest/)) 
implementation for the NWIS dataset. 

**A new tutorial notebook is available [here](https://github.com/gantian127/bmi_nwis/blob/master/notebooks/bmi_nwis.ipynb)**


## Quick Start Tutorial 

This notebook will help you get started using the nwis package to download the National Water Information System (NWIS) datasets. 

This tutorial includes the following sections:

1. [Brief Introduction](#section1)

   This section provides basic information about nwis package. 
   <br>
   
2. [Start with Examples](#section2)
   
   This section provides two examples to demonstrate how to use nwis package to download datasets for visualization.
   <br>

<a id='section1'></a>
## 1. Brief Introduction

**nwis package is deprecated.**
nwis package provides a set of functions that allows downloading of the observational datasets from the National Water Information System ([NWIS](https://waterdata.usgs.gov/nwis?)) for data analysis and visualization. 


To install nwis package, you can download the source code and use the following command:
```
$ pip install -e .
```

<a id='section2'></a>
## 2. Start with Examples

**Please note that the Nwis and BmiNwis classes in nwis package are no longer in use.** 

**Please check the new BmiNwis implementation from the [bmi_nwis]() package.**

In nwis package, Nwis class is designed for users to download datasets. BmiNwis class is designed to convert NWIS dataset as a data component for the [PyMT](https://pymt.readthedocs.io/en/latest/?badge=latest) modeling framework. The following examples demonstrate how to download the same dataset using Nwis and BmiNwis for data visualization. 

### Example 1: use Nwis class to download data (Recommended method)

Import Nwis class and download data with **get_data( )** method. This example downloads datasets for discharge, gage height, and water temperature observed at the USGS gage [03339000](https://waterdata.usgs.gov/nwis/uv?site_no=03339000) from Jan 1st to 15th, 2020. You can check the details of the [parameter settings](https://nwis.readthedocs.io/en/latest/#parameter-settings) for get_data( ) method to better understand the parameter values used in the example.

In [ ]:
import matplotlib.pyplot as plt

from nwis import Nwis

# get data from NWIS
nwis_data = Nwis()
dataset = nwis_data.get_data(
    site="03339000", start_date="2020-01-01", end_date="2020-01-15", data_type="dv"
)

The downloaded NWIS dataset is stored as a self-described xarray dataset object. With this dataset object, you can check the metadata and make time series plot.

In [ ]:
# show site metadata
dataset.attrs

# show data variable metadata
for var_name in dataset.data_vars:
    print(dataset[var_name].attrs)

In [ ]:
# plot discharge data
dataset["00060"].plot(figsize=(12, 7))
plt.ylabel(
    "{} ({})".format(dataset["00060"].variable_name, dataset["00060"].variable_unit)
)
plt.title("Discharge Observation at USGS Gage 03339000")

### Example 2: use BmiNwis class to download data (Demonstration of how to use BMI)

Import BmiNwis class and instantiate it. A configuration file (yaml file) is required to provide the parameter settings for data download. An example configure_file.yaml file is provided in the same folder with this Jupyter Notebook file. 

In [ ]:
import cftime
import matplotlib.pyplot as plt
import numpy as np

from nwis import BmiNwis

# initiate a data component
data_comp = BmiNwis()
data_comp.initialize("config_file.yaml")

Use variable related methods from BmiNwis class to check the variable information of the NWIS dataset. This data component stores multiple variables, including discharge, gage height, and water temperature. 

In [ ]:
# get variable info
for var_name in data_comp.get_output_var_names():
    var_unit = data_comp.get_var_units(var_name)
    print(f" variable_name: {var_name}\n var_unit: {var_unit}\n")

Use time related methods of BmiNwis class to check the time information of the NWIS dataset. The time values are stored in a format which follows [CF convention](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.8/cf-conventions.pdf). 

In [ ]:
# get time info
start_time = data_comp.get_start_time()
end_time = data_comp.get_end_time()
time_step = data_comp.get_time_step()
time_unit = data_comp.get_time_units()
time_steps = int((end_time - start_time) / time_step) + 1
print(
    " start_time:{}\n end_time:{}\n time_step:{}\n time_unit:{}\n time_steps:{}\n".format(
        start_time, end_time, time_step, time_unit, time_steps
    )
)

Loop through each time step to get the discharge and time values. discharge_array stores discharge values. cftime_array stores the numerical time values. time_array stores the corresponding Python datetime objects. get_value( ) method returns the discharge value at each time step. update( ) method updates the current time step of the data component.

In [ ]:
# initiate numpy arrays to store discharge data
discharge_value = np.empty(1)
discharge_array = np.empty(time_steps)
cftime_array = np.empty(time_steps)

for i in range(0, time_steps):
    data_comp.get_value("discharge", discharge_value)
    discharge_array[i] = discharge_value
    cftime_array[i] = data_comp.get_current_time()
    data_comp.update()

time_array = cftime.num2date(
    cftime_array,
    time_unit,
    only_use_cftime_datetimes=False,
    only_use_python_datetimes=True,
)

In [ ]:
# plot discharge data
plt.figure(figsize=(9, 5))
plt.plot(time_array, discharge_array)
plt.ylabel("{} ({})".format("discharge", "cubic feet per second"))
plt.title("Discharge Observation at USGS Gage 03339000")